In [ ]:
# 09.09.2022
# FInd the value of the hydration model coefficents by performing scipy optimise

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use({'figure.facecolor':'white'})
import matplotlib as mpl
from matplotlib.patches import Rectangle
from matplotlib import rc
from scipy.optimize import minimize
from matplotlib import cm, ticker
mpl.rcParams['font.size'] = 16
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'

#mpl.rcParams['font.family'] = ['times new roman'] # default is sans-serif
#rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=False)
mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath,bm}'] #for \text command

import scipy.stats as ss
from tqdm import tqdm
from datetime import datetime
now = datetime.now()
date = now.strftime("%d_%m_%Y_%H:%M")
import torch as th
import seaborn as sns
from mpl_toolkits import mplot3d
import fenics_concrete
import yaml
# local imports


# Initialize random number generator
RANDOM_SEED = 420
rng = np.random.default_rng(RANDOM_SEED)
%load_ext autoreload
%autoreload 2
seed = 420

In [ ]:
data_file = '../artificial_hydration_data/artificial_hydration_data.yaml'
#Example 1:
# read file and access artificial data:
with open(data_file) as file:
    hydration_data = yaml.safe_load(file)

# data is given in dictionary
# data[mix ratio: 0/.2/0.5/.8/1][temperature: 20/40/60][time/heat]
# it is assumed that this is hydration data for two distinct mixes
# mix 1: mix ration = 0 and mix 2: mix ratio = 1
# there are 3 intermediate mixes with 20/80, 50/50 and 80/20 ratio between mix 1 and 2
# for each of the 5 mixes there are 3 temperature measurements, each at 20, 40 and 60 degree
# for each temperature there is a list with the time and the heat values

# loop over all data, print lists
for mix_r in hydration_data:
    for temp in hydration_data[mix_r]:
        print(mix_r,temp,'time:',hydration_data[mix_r][temp]['time'])
        print(mix_r,temp,'heat:',hydration_data[mix_r][temp]['heat'])

In [ ]:
def forward_model(inp_latents: list, inp_obs: dict) -> list:
    parameter = fenics_concrete.Parameters()  # using the current default values

    # -- latents -----
    # parameter['B1'] = 2.916E-4  # in 1/s (le 0, < 0.1)
    # parameter['B2'] = 0.0024229  # - (le 0, smaller 1)
    # parameter['eta'] = 5.554  # something about diffusion (should be larger 0)
    # parameter['T_ref'] = 25  # reference temperature in degree celsius
    # parameter['Q_pot'] = 500e3 # potential heat per weight of binder in J/kg

    parameter['B1'] = inp_latents[0]  # in 1/s (le 0, < 0.1)
    parameter['B2'] = inp_latents[1] # - (le 0, smaller 1)
    parameter['eta'] = inp_latents[2]  # something about diffusion (should be larger 0)
    parameter['Q_pot'] = inp_latents[3] # potential heat per weight of binder in J/kg

    # -- observed inputs
    parameter['igc'] = 8.3145  # ideal gas constant in [J/K/mol], CONSTANT!!!
    parameter['zero_C'] = 273.15  # in Kelvin, CONSTANT!!!
    parameter['E_act'] = 47002  # activation energy in Jmol^-1 (no relevant limits) (Depends only on simulated temp, if that is not change no need to infer E_act)
    parameter['alpha_max'] = 0.875  # also possible to approximate based on equation with w/c (larger 0 and max 1)
    parameter['T_ref'] = 25  # reference temperature in degree celsius

    # this is the minimal time step used in the simulation
    # using a larger value will increase the speed but decrease the accuracy
    dt = 300 # value in seconds

    # this is the simulated temperature, needs to be adjusted depending on the temperature of the experimental data
    T = inp_obs['T_rxn'] # can be 20,40,60 as pert the exp values
    # this is the list of measured time data as given by the experiments
    #time_list = [0,5000,10000,20000,100000]
    time_list = inp_obs['time_list']

    # initiate material problem, for this the "fenics_concrete" conda package needs to be installed
    # use: 'mamba install -c etamsen fenics_concrete"
    problem = fenics_concrete.ConcreteThermoMechanical()

    # get the hydration function
    # this might change in the future to make it more easily accessible but for now it should work like this
    hydration_fkt = problem.get_heat_of_hydration_ftk()
    # the results are a heat list and a degree of hydration list, which you can ignore for now
    heat_list, doh_list= hydration_fkt(T, time_list, dt, parameter)

    return heat_list